You can download and run this notebook locally, or you can run it for free in a cloud environment using Colab or Sagemaker Studio Lab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kirbyju/TCIA_Notebooks/blob/main/TCIA_Aspera_CLI_Downloads.ipynb)

[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github.com/kirbyju/TCIA_Notebooks/blob/main/TCIA_Aspera_CLI_Downloads.ipynb)

# Summary
Much of non-DICOM content in [The Cancer Imaging Archive (TCIA)](https://www.cancerimagingarchive.net/) is provided via links to IBM Aspera Faspex packages.  Aspera’s FASP protocol is designed to move data rapidly across networks with minimal disruption to other traffic.  Aspera’s Faspex application bundles data into packages that can be referenced via a web link (i.e., a URI).  When an Aspera Faspex link resolves in a browser, it presents a GUI that guides a user through the process of installing a browser extension and a local Aspera Connect client, if not already there, that moves the data using FASP between TCIA servers and the computer the browser is running on.
We frequently get requests from researchers for an option to download TCIA Faspex packages using a command line interface, suitable for inclusion in shell/batch scripts, that bypasses the GUI.  While not a part of the standard Aspera distributions, the IBM Aspera developers have provided an open source tool (Apache 2.0 license) called ascli (aspera-cli) that allows a client to download an Aspera Faspex package using its URI.


# Install prerequisite software

In [1]:
!apt-get install ruby-dev
!sudo apt install libtool libffi-dev ruby ruby-dev make
!sudo apt install libzmq3-dev libczmq-dev
!gem install ffi-rzmq rake
!gem install specific_install
!gem specific_install https://github.com/SciRuby/iruby
!iruby register
!gem install net-protocol -v 0.1.2
!gem install net-smtp -v 0.3.0
!gem install aspera-cli
!ascli conf ascp install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-lato javascript-common libgmp-dev libgmpxx4ldbl libjs-jquery
  libruby2.5 rake ruby ruby-did-you-mean ruby-minitest ruby-net-telnet
  ruby-power-assert ruby-test-unit ruby2.5 ruby2.5-dev ruby2.5-doc
  rubygems-integration
Suggested packages:
  apache2 | lighttpd | httpd gmp-doc libgmp10-doc libmpfr-dev ri bundler
The following NEW packages will be installed:
  fonts-lato javascript-common libgmp-dev libgmpxx4ldbl libjs-jquery
  libruby2.5 rake ruby ruby-dev ruby-did-you-mean ruby-minitest
  ruby-net-telnet ruby-power-assert ruby-test-unit ruby2.5 ruby2.5-dev
  ruby2.5-doc rubygems-integration
0 upgraded, 18 newly installed, 0 to remove and 7 not upgraded.
Need to get 8,365 kB of archives.
After this

# Download an Aspera package

Now that the prerequisite installations are complete, you will be able to use the ascli command to transfer a TCIA Faspex package using just its URI. To get a TCIA packages’s URI:
1.	Browse to the collection landing page that describes the package, 
2.	Find the “Download” button that one normally would use to download the package using the Faspex GUI, but do not click it,
3.	Right click that button icon and select “copy link address” or “copy link” or similar (depends on your browser) to extract the package URI into the clipboard or copy/paste buffer.

Now that you have the package URI, the method for transferring a package to your system is with a command similar to the following, replacing the last line with the URI of the package you want to download.  You may also want to change the **--to-folder** path.

**Note:** If you don't change the package URL below it will take about 5 minutes to download an example package (~9 gbytes) from the [Comparison of mIF versus mIHC for immune markers in head and neck carcinoma (HNSCC-mIF-mIHC-comparison)](https://doi.org/10.7937/TCIA.2020.T90F-WB82) collection.


In [2]:
!ascli faspex package recv \
   --url=https://faspex.cancerimagingarchive.net/aspera/faspex \
   --username=none --password=none \
   --to-folder=asperaDownloads \
   https://faspex.cancerimagingarchive.net/aspera/faspex/external_deliveries/74?passcode=56970ac8fccfb0cdb926271492d7f3ab3ea8721c


Completed: 8750198K bytes transferred in 221 seconds
 (323824K bits/sec), in 865 files, 5 directories.
+--------------------------------------------------------------------------------------------------------------------------------+---------+
| package                                                                                                                        | status  |
+--------------------------------------------------------------------------------------------------------------------------------+---------+
| https://faspex.cancerimagingarchive.net/aspera/faspex/external_deliveries/74?passcode=56970ac8fccfb0cdb926271492d7f3ab3ea8721c | success |
+--------------------------------------------------------------------------------------------------------------------------------+---------+


# Acknowledgements
TCIA is funded by the [Cancer Imaging Program (CIP)](https://imaging.cancer.gov/), a part of the United States [National Cancer Institute (NCI)](https://www.cancer.gov/).  It is managed by the [Frederick National Laboratory for Cancer Research (FNLCR)](https://frederick.cancer.gov/) and hosted by the [University of Arkansas for Medical Sciences (UAMS)](https://www.uams.edu/)

This notebook was created by [Justin Kirby](https://www.linkedin.com/in/justinkirby82/), [Quasar Jarosz](https://www.linkedin.com/in/quasarjarosz/) and [Lawrence Tarbox](https://www.linkedin.com/in/lawrence-tarbox-088335/). If you leverage this notebook or any TCIA datasets in your work, please be sure to comply with the [TCIA Data Usage Policy](https://wiki.cancerimagingarchive.net/x/c4hF). In particular, make sure to cite the DOI(s) for the specific TCIA datasets you used in addition to the following paper!

## TCIA Citation

Clark, K., Vendt, B., Smith, K., Freymann, J., Kirby, J., Koppel, P., Moore, S., Phillips, S., Maffitt, D., Pringle, M., Tarbox, L., & Prior, F. (2013). The Cancer Imaging Archive (TCIA): Maintaining and Operating a Public Information Repository. Journal of Digital Imaging, 26(6), 1045–1057. https://doi.org/10.1007/s10278-013-9622-7